In [95]:
# Load dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, text
import pymc as pm
import arviz as az
from scipy.stats import zscore

# Define the SQLalchemy engine
engine = create_engine(f"sqlite:////Users/JO/PhD/neuro-ascertainment/data/db.sqlite")

# Read the SQL query from the file
with open('/Users/JO/PhD/neuro-ascertainment/candidate-queries/karolinska-nsicu-cohort/karolinska-cohort.sql', 'r') as file:
    query = file.read()

In [96]:
query_PAR_HADM = query + "SELECT * FROM PAR_HADM"
PAR_HADM = pd.read_sql(query_PAR_HADM, engine)

In [100]:
def count_id(df):
    LopNr = df['LopNr'].nunique() if 'LopNr' in df else 'Column missing'
    VtfId_LopNr = df['VtfId_LopNr'].nunique() if 'VtfId_LopNr' in df else 'Column missing'
    HADM_ID = df['HADM_ID'].nunique() if 'HADM_ID' in df else 'Column missing'
    return print(f'Unique patients: {LopNr} | Unique SIR admits: {VtfId_LopNr} | Unique PAR admits: {HADM_ID}')

In [101]:
count_id(PAR_HADM)

Unique patients: 59333 | Unique SIR admits: Column missing | Unique PAR admits: 359305


In [102]:
query_K_ICU_ADMISSIONS = query + "SELECT * FROM K_ICU_ADMISSIONS"
K_ICU_ADMISSIONS = pd.read_sql(query_K_ICU_ADMISSIONS, engine)

In [103]:
count_id(K_ICU_ADMISSIONS)

Unique patients: 6454 | Unique SIR admits: 7673 | Unique PAR admits: Column missing


In [104]:
query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR = query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR"
K_ICU_ADMISSIONS_MATCHED_WITH_PAR = pd.read_sql(query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR, engine)

In [105]:
count_id(K_ICU_ADMISSIONS_MATCHED_WITH_PAR)

Unique patients: 5649 | Unique SIR admits: 6498 | Unique PAR admits: 6621


In [106]:
query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX = query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX"
K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX = pd.read_sql(query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX, engine)

In [107]:
count_id(K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX)

Unique patients: 5649 | Unique SIR admits: 6498 | Unique PAR admits: 6621


In [108]:
query_DESCRIPTIVE = query + "SELECT * FROM DESCRIPTIVE "
DESCRIPTIVE  = pd.read_sql(query_DESCRIPTIVE, engine)

In [109]:
count_id(DESCRIPTIVE)

Unique patients: 5649 | Unique SIR admits: 6498 | Unique PAR admits: 6621


In [118]:
result = DESCRIPTIVE.groupby('DX_GROUP', dropna=False).agg({'DX_GROUP': 'size', 'age': 'mean', 'sex_female': 'mean', 'SAPS_GCS': 'median', 'd30': 'mean', 'any_AMV': 'mean', 'MVO': lambda x: x.mode().iloc[0] if not x.mode().empty else None})
result = result.rename(columns={'DX_GROUP': 'count'})
result

,count,age,sex_female,SAPS_GCS,d30,any_AMV,MVO
DX_GROUP,,,,,,,
ABM,245,55.648980,0.493878,12.0,0.159184,0.771429,121
AIS,552,62.289855,0.302536,10.0,0.257246,0.844203,221
ASAH,1408,57.762784,0.638494,14.0,0.127841,0.618608,331
AVM,45,47.422222,0.533333,15.0,0.044444,0.533333,331
CFX,300,60.833333,0.260000,15.0,0.106667,0.706667,331
CVT,23,43.956522,0.434783,11.0,0.173913,0.521739,221
ENC,44,46.659091,0.431818,10.0,0.068182,0.795455,121
HC,62,56.306452,0.580645,11.0,0.080645,0.822581,331
ICH,759,58.839262,0.405797,8.0,0.324111,0.849802,331


In [ ]:
MVO.